<a href="https://colab.research.google.com/github/kumarchandan/100-days-of-deep-learning/blob/master/day-14/Day_14_Fine_tuning_HyperParameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [10]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

print(X_train_full.shape)
print(X_test.shape)
print(y_train_full.shape)
print(y_test.shape)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
print('-------------Split into Train-Validation dataset--------------')
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)


scalar = StandardScaler()
X_train_tr = scalar.fit_transform(X_train)
X_valid_tr = scalar.transform(X_valid)
X_test_tr = scalar.transform(X_test)

(15480, 8)
(5160, 8)
(15480,)
(5160,)
-------------Split into Train-Validation dataset--------------
(11610, 8)
(3870, 8)
(11610,)
(3870,)


In [11]:
def build_model(n_hidden=2, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))

  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation='relu'))
  
  model.add(keras.layers.Dense(1))
  optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  model.compile(loss='mse', optimizer=optimizer)
  return model

In [12]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

- Now this object behaves like a regular `scikit-learn` regressor and it's methods can be called
 - `fit()`
 - `score()`
 - `predict()`

In [13]:
keras_reg.fit(X_train_tr, y_train, epochs=100, validation_data=(X_valid_tr, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_score = keras_reg.score(X_test_tr, y_test)
y_pred = keras_reg.predict(X_test_tr[:3])
print(y_pred)

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 0.9899 - val_loss: 8.3736
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5630 - val_loss: 2.8030
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4743 - val_loss: 0.4149
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4325 - val_loss: 0.3908
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4176 - val_loss: 0.4089
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4078 - val_loss: 0.4305
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3990 - val_loss: 0.4044
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3930 - val_loss: 0.3809
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3870 - val_loss: 0.3774
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3822 - val_lo

In [19]:
print(y_pred)
print(mse_score)

[0.71318156 1.5864167  4.281155  ]
-0.326223760843277


## Search the hyperparameter space

In [15]:
import numpy as np
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params_distribs = {
    'n_hidden': [0, 1, 2, 3, 4],
    'n_neurons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(
    estimator=keras_reg,
    param_distributions=params_distribs,
    n_iter=10,
    cv=3
)

rnd_search_cv.fit(X_train_tr, y_train, epochs=100, validation_data=(X_valid_tr, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
242/242 [==============================] - 0s 2ms/step - loss: 1.4669 - val_loss: 83.1100
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3348 - val_loss: 0.8100
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5761 - val_loss: 0.5082
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5034 - val_loss: 0.4622
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4655 - val_loss: 0.4339
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4449 - val_loss: 0.4173
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4317 - val_loss: 0.4052
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4213 - val_loss: 0.3949
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4141 - val_loss: 0.3886
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4090 - val_l

In [17]:
rnd_search_cv.best_params_

{'learning_rate': 0.024146164821179653, 'n_hidden': 4, 'n_neurons': 95}

In [18]:
rnd_search_cv.best_score_

-0.3022942543029785

In [21]:
rnd_search_cv

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fc859bc4828>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc84ecc1cf8>,
                                        'n_hidden': [0, 1, 2, 3, 4],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_